In [3]:
import cv2
import os
import h5py                                                                                                                                                                                   
import numpy as np

from keras import applications, optimizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.models import load_model
from sklearn.model_selection import train_test_split

%run Models.ipynb

## Load image

In [ ]:
def Load_Image(path):
    pass

## Define Loss functions

In [ ]:
def MSE_Loss():
    pass

In [ ]:
## 回傳兩個
def Adversial_Loss():
    pass

In [ ]:
def Constrained_Loss():
    pass

In [ ]:
## 兩種模式
def GN_Loss(mode):
    switcher = {
        1: 
        2:
    }
    pass

In [ ]:
def DN_Loss():
    pass

## Training

In [4]:
# Step 1: 讀取訓練資料（受損 / 原圖）
path_dec = ""  ## 受損影像路徑
path_ori = ""  ## 原圖影像路徑

data_dec = Load_Image(path_dec)
data_ori = Load_Image(path_ori)


# Step 2: 讀取 Models
GN = Get_GN_train()
DN = Get_DN_train()


# Step 3: 先稍微訓練 Generative Network


# Step 4: 加入 Discriminiative Network 一起訓練， 儲存權重


# Step 5: 儲存 Final 權重

NameError: name 'Load_Image' is not defined